In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from glider import Glider
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
import pickle

# from stable_baselines3.common.env_checker import check_env

In [ ]:
glider = Glider(u0=0.25, v0=0.25, w0=0.1)

checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path="./big_state_models/",
    name_prefix="rl_model",
)

In [ ]:
glider.beta, glider.u, glider.v, glider.w, glider.x, glider.y, glider.theta

In [ ]:
glider.step(1)

In [ ]:
len(glider.u)

In [ ]:
model = PPO("MlpPolicy", glider, verbose=1, tensorboard_log="big_state_logs/")
model.learn(total_timesteps=6e5, callback=checkpoint_callback)

In [ ]:
glider = Glider(u0=0.25, v0=-0.25, w0=0.1)
model = PPO.load("big_state_models/rl_model_600000_steps.zip", env=glider)
done = False
obs = glider.reset()
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, _ = glider.step(action.item())

In [ ]:
plt.plot(glider.theta)

In [ ]:
plt.scatter(glider.x, glider.y)

In [ ]:
plt.plot(glider.beta)

In [ ]:
def save_history(glider: Glider, filename: str) -> None:
    """
    Save the full history of the glider in a dictionary
    """
    history = {
        "u": glider.u,
        "v": glider.v,
        "w": glider.w,
        "x": glider.x,
        "y": glider.y,
        "theta": glider.theta,
        "beta": glider.beta,
    }
    with open(filename + ".pkl", "wb") as f:
        pickle.dump(history, f)

In [ ]:
save_history(glider=glider, filename="history")

In [ ]:
# model = PPO("MlpPolicy", env=glider, verbose=1)
# obs = glider.reset()
# done = False
# while not done:
#     action, _states = model.predict(obs)
#     obs, rewards, done, info = glider.step(action.item())